In [18]:
import numpy as np 
import pandas as pd
import plotly.express as px 
import sklearn
import torch
import time

In [73]:
df_train = pd.read_csv('../input/fashion-mnist_train.csv')
df_test = pd.read_csv('../input/fashion-mnist_test.csv')
X_train = df_train.drop('label', axis=1).to_numpy().reshape((60000, 28, 28))
X_test = df_test.drop('label', axis=1).to_numpy().reshape((10000, 28, 28))
y_train = pd.get_dummies(df_train['label']).rename(columns={0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}).to_numpy()
y_test = pd.get_dummies(df_test['label']).rename(columns={0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}).to_numpy()

In [67]:
def adjacent_pixels(pixel_cord: tuple):
    x = pixel_cord[0]
    y = pixel_cord[1]
    temp_matrix = [[(max(0, x - 1), max(0, y - 1)), (max(0, x - 1), y), (max(0, x - 1), min(27, y + 1))], [(x, max(0, y - 1)), pixel_cord, (x, min(27, y + 1))], [(min(27, x + 1), max(0, y - 1)), (min(27, x + 1), y), (min(27, x + 1), min(27, y + 1))]]
    cord_matrix = []
    for layer in temp_matrix:
        for cord in layer:
            if cord not in cord_matrix:
                cord_matrix.append(cord)
    return cord_matrix
    

In [101]:
image = np.zeros((28, 28))

In [102]:
pixel_selection = (27, 0)
arr = adjacent_pixels(pixel_selection)
for i in arr:
    image[i[0]][i[1]] = 150
image[pixel_selection[0]][pixel_selection[1]] = 250

In [103]:
px.imshow(image)